# FIFA Players Classification

This notebook contains the code to train models for the FIFA Classification problem based on their attributes

## Step 1: Import libraries

Libraries we will be using:
- **tensorflow** and **keras**: APIs for neural network model construction and training
- **numpy**: Numerical python library.
- **pandas**: Will be used to work with dataframes from .csv files.
- **sklearn**: Will be used to plot the confusion matrix.
- **matplotlib**: Will be used to plot graphics.
- **time**: To measure training time.
- **datetime**: To get the datetime a new model is created and use that to create its name

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import LearningRateScheduler,TerminateOnNaN, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime

## Step 2: Load data

We first define the path to our data

In [2]:
DATA_FOLDER = "../Data"

ATT_FILE = f"{DATA_FOLDER}/FootballPlayerPreparedCleanAttributes.csv"
IMPUTED_ATT_FILE = f"{DATA_FOLDER}/ImputedFootballPlayerPreparedCleanAttributes.csv"

ONE_HOT_LABEL_FILE = f"{DATA_FOLDER}/FootballPlayerOneHotEncodedClasses.csv"
IMPUTED_ONE_HOT_LABEL_FILE = f"{DATA_FOLDER}/ImputedFootballPlayerOneHotEncodedClasses.csv"

Specify training options:
- **IMPUTED_DATA**: We will use file with data imputed
- **TRAIN_RATE**: The amount of instances to use during training, it will also determine the dev and test instances number indirectly

In [3]:
IMPUTED_DATA = False    # Use imputed dataset
TRAIN_RATE = 0.8        # The division of the data for training, validation and test

Because our data has more than 10000, we will be using 80-10-10 split in our dataset:
- **80%** for training
- **10%** for validation/dev test
- **10%** for final test

In [4]:
if IMPUTED_DATA:
    attributes = pd.read_csv(IMPUTED_ATT_FILE)
    labels = pd.read_csv(IMPUTED_ONE_HOT_LABEL_FILE)
else:
    attributes = pd.read_csv(ATT_FILE)
    labels = pd.read_csv(ONE_HOT_LABEL_FILE)

n_total_instances = attributes.shape[0]
n_train_instances = int(n_total_instances * TRAIN_RATE)
n_dev_instances = int((n_total_instances - n_train_instances)/2)

instances_train = attributes.values[:n_train_instances]
labels_train = labels.values[:n_train_instances]

instances_dev = attributes.values[n_train_instances: n_train_instances + n_dev_instances]
labels_dev = labels.values[n_train_instances: n_train_instances + n_dev_instances]

print("instances_train:", instances_train.shape)
print("labels_train:", labels_train.shape)
print("instances_dev:", instances_dev.shape)
print("labels_dev:",labels_dev.shape)

instances_train: (12897, 17)
labels_train: (12897, 4)
instances_dev: (1612, 17)
labels_dev: (1612, 4)


## Step 3: Initialize variables
- **INPUT_SIZE**: The size of the training instances
- **OUTPUT_SIZE**: The number of possible classes
- **NUM_TRAINING_EXAMPLES**: The number of possible classes
- **NUM_DEV_EXAMPLES**: The number of validation/dev test instances

In [5]:
INPUT_SIZE = instances_train.shape[1]
print(INPUT_SIZE)
OUTPUT_SIZE = labels_train.shape[1]
print(OUTPUT_SIZE)
NUM_TRAINING_EXAMPLES = int(round(instances_train.shape[0]/1))
NUM_DEV_EXAMPLES = int(round(instances_dev.shape[0]/1))

17
4


## Step 4: Set hyperparameters

- **LEARNING_RATE** $\alpha$: The step size in the learning process
- **BATCH_SIZE**: The size of each batch
- **N_HIDDEN**: Number of neurons in the hidden.

The learning process stops when it reaches 20,000 learning iterations. An iteration comprises presenting the network *batch_size* learning examples and then adjusting the weights (parameters).

In [6]:
N_EPOCHS = 5000
LEARNING_RATE = 0.01
BATCH_SIZE = 4096
ACTIVATION = "relu"
N_HIDDEN = [32, 16, 8]
REGULARIZER = keras.regularizers.L1L2(l1=0.001,l2=0.1)
INITIALIZER = keras.initializers.he_normal(seed=None)
LOSS = tf.keras.losses.categorical_crossentropy
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
METRICS = ["categorical_accuracy"]

## Step 5: Build the model

In [7]:
def architecture1():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,), batch_size=None))
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons, activation=ACTIVATION))
    model.add(keras.layers.Dense(OUTPUT_SIZE, activation="softmax"))
    
    model_name = "D512"
    return model, model_name


def architecture2():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     activation=ACTIVATION,
                                     kernel_regularizer=REGULARIZER
                                    ))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE, 
                                 activation="softmax"))
    
    model_name = "D128_D64_D32"
    return model, model_name

def architecture3():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    model_name = "D128_BN_D64_BN_D32_BN"
    return model, model_name

def architecture4():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeUniform(seed=None),
                                     use_bias=False
                                     ))
    
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    model_name = "D2048_BN_D1024_BN_D512_BN_D256_BN_D128_BN_D64_BN_D32_BN"
    return model, model_name


def architecture5():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
    
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    
    model_name = "D512_BN_D128_BN_D32_BN_D128_BN_D512_BN"
    return model, model_name

def architecture5dr():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
        model.add(keras.layers.Dropout(rate=0.1))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    
    model_name = "D512_DR_BN_D128_DR_BN_D32_DR_BN_D128_DR_BN_D512_DR_BN"
    return model, model_name


def final_architecture():
    model = keras.Sequential(name="Model")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="softmax"
                                ))
    
    model_name = "final"
    return model, model_name
    
    
def compile_model(model):
    my_model.compile(loss=LOSS,
                     optimizer=OPTIMIZER,
                     metrics=METRICS)

In [8]:
my_model, model_name = final_architecture()

In [9]:
compile_model(my_model)

## Step 6: Train the model

In [ ]:
checkpoint_filepath = "../Models/finalsin.h5"
reduce_lr = ReduceLROnPlateau('val_categorical_accuracy', factor=0.1, patience=150, verbose=1)
early_stop = EarlyStopping('val_categorical_accuracy', patience=500, verbose=1)
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,monitor='val_categorical_accuracy',mode='max',save_best_only=True)
callbacks = [reduce_lr,checkpoint]#,reduce_lr]
start_time = time.perf_counter()
history = my_model.fit(instances_train, 
                       labels_train, 
                       batch_size = BATCH_SIZE, 
                       epochs=N_EPOCHS, 
                       validation_data = (instances_dev, labels_dev),
                       callbacks=callbacks,
                       verbose = 1)

end_time = time.perf_counter()
best_idx = int(np.argmax(history.history['val_categorical_accuracy']))
best_value = np.max(history.history['val_categorical_accuracy'])
print('Best validation model: epoch ' + str(best_idx+1), ' - val_categorical_accuracy ' + str(best_value))

Epoch 1/5000
4/4 [==============================] - 2s 96ms/step - loss: 12.4669 - categorical_accuracy: 0.2555 - val_loss: 10.7076 - val_categorical_accuracy: 0.3046 - lr: 0.0100
Epoch 2/5000
4/4 [==============================] - 0s 18ms/step - loss: 10.1348 - categorical_accuracy: 0.3978 - val_loss: 8.8618 - val_categorical_accuracy: 0.2667 - lr: 0.0100
Epoch 3/5000
4/4 [==============================] - 0s 18ms/step - loss: 8.2811 - categorical_accuracy: 0.4413 - val_loss: 7.3089 - val_categorical_accuracy: 0.2581 - lr: 0.0100
Epoch 4/5000
4/4 [==============================] - 0s 19ms/step - loss: 6.7404 - categorical_accuracy: 0.4759 - val_loss: 6.0294 - val_categorical_accuracy: 0.2680 - lr: 0.0100
Epoch 5/5000
4/4 [==============================] - 0s 26ms/step - loss: 5.4762 - categorical_accuracy: 0.5207 - val_loss: 5.0006 - val_categorical_accuracy: 0.3263 - lr: 0.0100
Epoch 6/5000
4/4 [==============================] - 0s 25ms/step - loss: 4.4506 - categorical_accuracy: 0.5

Epoch 47/5000
4/4 [==============================] - 0s 20ms/step - loss: 0.4872 - categorical_accuracy: 0.8051 - val_loss: 1.9803 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 48/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4929 - categorical_accuracy: 0.8007 - val_loss: 1.9903 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 49/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4957 - categorical_accuracy: 0.8017 - val_loss: 2.0127 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 50/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4839 - categorical_accuracy: 0.8087 - val_loss: 2.0377 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 51/5000
4/4 [==============================] - 0s 18ms/step - loss: 0.5065 - categorical_accuracy: 0.7964 - val_loss: 2.0246 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 52/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4871 - categorical_accuracy: 

Epoch 93/5000
4/4 [==============================] - 0s 18ms/step - loss: 0.4776 - categorical_accuracy: 0.8101 - val_loss: 1.9782 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 94/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4736 - categorical_accuracy: 0.8114 - val_loss: 1.9957 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 95/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4829 - categorical_accuracy: 0.8035 - val_loss: 2.0809 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 96/5000
4/4 [==============================] - 0s 19ms/step - loss: 0.4703 - categorical_accuracy: 0.8127 - val_loss: 2.1241 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 97/5000
4/4 [==============================] - 0s 18ms/step - loss: 0.4696 - categorical_accuracy: 0.8110 - val_loss: 2.0946 - val_categorical_accuracy: 0.2339 - lr: 0.0100
Epoch 98/5000
4/4 [==============================] - 0s 18ms/step - loss: 0.4863 - categorical_accuracy: 

4/4 [==============================] - 0s 23ms/step - loss: 0.4693 - categorical_accuracy: 0.8119 - val_loss: 1.4179 - val_categorical_accuracy: 0.5099 - lr: 0.0100
Epoch 139/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.4730 - categorical_accuracy: 0.8096 - val_loss: 1.2663 - val_categorical_accuracy: 0.5744 - lr: 0.0100
Epoch 140/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4758 - categorical_accuracy: 0.8097 - val_loss: 1.0880 - val_categorical_accuracy: 0.5695 - lr: 0.0100
Epoch 141/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.4783 - categorical_accuracy: 0.8064 - val_loss: 0.9461 - val_categorical_accuracy: 0.6185 - lr: 0.0100
Epoch 142/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.4825 - categorical_accuracy: 0.8033 - val_loss: 1.0222 - val_categorical_accuracy: 0.5893 - lr: 0.0100
Epoch 143/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.4940 - categorical_accuracy: 0.7992 - 

Epoch 184/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4924 - categorical_accuracy: 0.7965 - val_loss: 0.8859 - val_categorical_accuracy: 0.6427 - lr: 0.0100
Epoch 185/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4787 - categorical_accuracy: 0.8047 - val_loss: 0.7614 - val_categorical_accuracy: 0.7010 - lr: 0.0100
Epoch 186/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4707 - categorical_accuracy: 0.8117 - val_loss: 0.5703 - val_categorical_accuracy: 0.7525 - lr: 0.0100
Epoch 187/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4807 - categorical_accuracy: 0.8039 - val_loss: 0.5522 - val_categorical_accuracy: 0.7630 - lr: 0.0100
Epoch 188/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4797 - categorical_accuracy: 0.8096 - val_loss: 0.5625 - val_categorical_accuracy: 0.7643 - lr: 0.0100
Epoch 189/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4849 - categorical_accu

...

4/4 [==============================] - 0s 24ms/step - loss: 0.3939 - categorical_accuracy: 0.8335 - val_loss: 0.4411 - val_categorical_accuracy: 0.8133 - lr: 1.0000e-11
Epoch 1997/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3953 - categorical_accuracy: 0.8344 - val_loss: 0.4411 - val_categorical_accuracy: 0.8133 - lr: 1.0000e-11
Epoch 1998/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3946 - categorical_accuracy: 0.8352 - val_loss: 0.4411 - val_categorical_accuracy: 0.8133 - lr: 1.0000e-11
Epoch 1999/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3957 - categorical_accuracy: 0.8326 - val_loss: 0.4411 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-11
Epoch 2000/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3961 - categorical_accuracy: 0.8322 - val_loss: 0.4411 - val_categorical_accuracy: 0.8127 - lr: 1.0000e-11
Epoch 2001/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3945 - catego

4/4 [==============================] - 0s 24ms/step - loss: 0.3940 - categorical_accuracy: 0.8338 - val_loss: 0.4411 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-11
Epoch 2041/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3954 - categorical_accuracy: 0.8319 - val_loss: 0.4411 - val_categorical_accuracy: 0.8127 - lr: 1.0000e-11
Epoch 2042/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.4016 - categorical_accuracy: 0.8313 - val_loss: 0.4411 - val_categorical_accuracy: 0.8127 - lr: 1.0000e-11
Epoch 2043/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3939 - categorical_accuracy: 0.8336 - val_loss: 0.4412 - val_categorical_accuracy: 0.8127 - lr: 1.0000e-11
Epoch 2044/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3949 - categorical_accuracy: 0.8333 - val_loss: 0.4411 - val_categorical_accuracy: 0.8114 - lr: 1.0000e-11
Epoch 2045/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3938 - catego

4/4 [==============================] - 0s 24ms/step - loss: 0.3951 - categorical_accuracy: 0.8324 - val_loss: 0.4412 - val_categorical_accuracy: 0.8145 - lr: 1.0000e-12
Epoch 2084/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3965 - categorical_accuracy: 0.8322 - val_loss: 0.4412 - val_categorical_accuracy: 0.8145 - lr: 1.0000e-12
Epoch 2085/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3946 - categorical_accuracy: 0.8337 - val_loss: 0.4412 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-12
Epoch 2086/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3963 - categorical_accuracy: 0.8334 - val_loss: 0.4412 - val_categorical_accuracy: 0.8127 - lr: 1.0000e-12
Epoch 2087/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3938 - categorical_accuracy: 0.8334 - val_loss: 0.4412 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-12
Epoch 2088/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3955 - catego

4/4 [==============================] - 0s 23ms/step - loss: 0.3946 - categorical_accuracy: 0.8314 - val_loss: 0.4410 - val_categorical_accuracy: 0.8139 - lr: 1.0000e-12
Epoch 2128/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3942 - categorical_accuracy: 0.8339 - val_loss: 0.4411 - val_categorical_accuracy: 0.8139 - lr: 1.0000e-12
Epoch 2129/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3965 - categorical_accuracy: 0.8310 - val_loss: 0.4411 - val_categorical_accuracy: 0.8133 - lr: 1.0000e-12
Epoch 2130/5000
4/4 [==============================] - 0s 23ms/step - loss: 0.3946 - categorical_accuracy: 0.8325 - val_loss: 0.4411 - val_categorical_accuracy: 0.8139 - lr: 1.0000e-12
Epoch 2131/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3944 - categorical_accuracy: 0.8328 - val_loss: 0.4411 - val_categorical_accuracy: 0.8133 - lr: 1.0000e-12
Epoch 2132/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3965 - catego

4/4 [==============================] - 0s 24ms/step - loss: 0.3972 - categorical_accuracy: 0.8314 - val_loss: 0.4412 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-12
Epoch 2172/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3950 - categorical_accuracy: 0.8321 - val_loss: 0.4411 - val_categorical_accuracy: 0.8120 - lr: 1.0000e-12
Epoch 2173/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3938 - categorical_accuracy: 0.8335 - val_loss: 0.4411 - val_categorical_accuracy: 0.8114 - lr: 1.0000e-12
Epoch 2174/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3949 - categorical_accuracy: 0.8324 - val_loss: 0.4411 - val_categorical_accuracy: 0.8114 - lr: 1.0000e-12
Epoch 2175/5000
4/4 [==============================] - 0s 25ms/step - loss: 0.3944 - categorical_accuracy: 0.8314 - val_loss: 0.4411 - val_categorical_accuracy: 0.8114 - lr: 1.0000e-12
Epoch 2176/5000
4/4 [==============================] - 0s 24ms/step - loss: 0.3953 - catego

## Step 7: Analyze results

In [ ]:
results=pd.DataFrame(history.history)
results.plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel ("Epochs")
plt.ylabel ("Accuracy - Mean Log Loss")
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
training_accuracy = round(results.categorical_accuracy.values[-1:][0]*100, 2)
val_accuracy = round(results.val_categorical_accuracy.values[-1:][0]*100, 2)
training_error = round((1 - results.categorical_accuracy.values[-1:][0])*100, 2)
test_error = round((1 - results.val_categorical_accuracy.values[-1:][0])*100, 2)
human_error = 5
bias = round(training_error - human_error, 2)
variance = round(test_error - training_error, 2)
training_time = round(end_time - start_time)
print("Training accuracy: ", training_accuracy, "%")
print("Validationa accuracy", val_accuracy, "%")
print("Error (training):", training_error, "%")
print("Error (test):", test_error, "%")
print("Bias:", bias)
print("Variance:", variance)
print("Time: ", training_time, "seconds")

In [ ]:
def draw_confusion_matrix(cm, categories, normalize=True):
    # Draw confusion matrix
    fig = plt.figure(figsize=[3*pow(len(categories), 0.5), 2*pow(len(categories), 0.5)])
    ax = fig.add_subplot(111)
    if normalize:
        cm = cm.astype('float') / np.maximum(cm.sum(axis=1)[:, np.newaxis], np.finfo(np.float64).eps)
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.get_cmap('Blues'))
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), xticklabels=categories, yticklabels=categories, ylabel='Annotation', xlabel='Prediction')
    # Rotate the tick labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    # Loop over data dimensions and create text annotations
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), ha="center", va="center", color="white" if cm[i, j] > thresh else "black", fontsize=int(12-pow(len(categories), 0.5)))
    fig.tight_layout()
    plt.show(fig)


In [ ]:
labels_pred = my_model.predict(instances_dev)
cm = confusion_matrix(np.argmax(labels_dev, axis=1), np.argmax(labels_pred, axis=1))
draw_confusion_matrix(cm, ["Poor", "Intermediate", "Good", "Excellent"])

## Step 8: Save results

In [ ]:
RESULTS_FOLDER = "../Results"
RESULTS_FILE = f"{RESULTS_FOLDER}/models_performance.csv"

In [ ]:
results_df = pd.read_csv(RESULTS_FILE, index_col = 0)

In [ ]:
COLUMNS = [
    'model_name',
    'epochs',
    'learning_rate',
    'batch_size',
    'activation',
    'loss',
    'optimizer',
    'regularization',
    'initializer',
    'train_accuracy',
    'validation_accuracy',
    'bias',
    'variance',
    'training_time'
]

new_result = [[
    model_name,
    N_EPOCHS,
    LEARNING_RATE,
    BATCH_SIZE,
    ACTIVATION,
    "categorical_crossentropy",
    "ADAM 0.9 0.999",
    "L1L2",
    "He Uniform",
    training_accuracy,
    val_accuracy,
    bias,
    variance,
    training_time]]

new_result_df = pd.DataFrame(new_result, columns=COLUMNS)
concatenation = pd.concat([results_df, new_result_df], ignore_index=True)
concatenation.to_csv(RESULTS_FILE)


In [ ]:
# last_five = results_df.tail(5)
last_five = concatenation.tail(5)
print(last_five)
print("\n---Metrics---")
print(last_five[["train_accuracy", "validation_accuracy", "bias", "variance", "training_time"]].describe())

In [ ]:
MODELS_FOLDER = "../Models"
save_name = f"{MODELS_FOLDER}/{model_name}-2.h5"
my_model.save(save_name)